In [7]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [8]:
import tensorflow as tf

In [9]:
slim = tf.contrib.slim
trunc_normal = lambda stddev: tf.truncated_normal_initializer(0.0,stddev)

In [10]:
def alexnet_scope(weight_decay=0.0005):
    with slim.arg_scope([slim.conv2d,slim.fully_connected],
                       activation_fn = tf.nn.relu,
                       biases_initializer = tf.constant_initializer(0.1),
                       weights_regularizer=slim.l2_regularizer(weight_decay)):
        with slim.arg_scope([slim.conv2d],padding='SAME'):
            with slim.arg_scope([slim.max_pool2d],padding='VALID') as arg_sc:
                return arg_sc

def alex_net(inputs,
            num_classes=1000,
            is_training=True,
            dropout_keep_prob=0.5,
            spatial_squeeze=True,
            scope='alex_net'):
    with tf.variable_scope(scope, 'alex_net',[inputs]) as sc:
        end_point_collection = sc.name + '_end_points'
        with slim.arg_scope([slim.conv2d, slim.fully_connected, slim.max_pool2d],
                           output_collections=[end_point_collections]):
            net = slim.conv2d(inputs, 64,[11,11], 4, padding='VALID',scope='conv1')
            net = slim.max_pool2d(net,[3,3],2,scope='pool1')
            net = slim.conv2d(net, 192,[5,5], scope='conv2')
            net = slim.max_pool2d(net,[3,3],2,scope='pool2')
            net = slim.conv2d(net, 384, [3,3],scope='conv3')
            net = slim.conv2d(net, 384, [3,3],scope='conv4')
            net = slim.conv2d(net, 256, [3,3],scope='conv5')
            net = slim.max_pool2d(net,[3,3],2,scope='pool3')
            
            with slim.arg_scope([slim.conv2d],
                               weight_initializer=trunc_normal(0.005),
                               biases_initializer=tf.constant_intializer(0.1)):
                net = slim.conv2d(net, 4096,[5,5], padding='VALID',scope='fc6')
                net = slim.dropout(net, dropout_keep_prob, is_training=is_training,scope='dropout6')
                net = slim.conv2d(net, 4096,[1,1], scope='fc7')
                net = slim.dropout(net, dropout_keep_prob, is_training=is_training,scope='dropout7')
                net = slim.conv2d(net, num_classes,[1,1],
                                 activation_fn=None,
                                 initializer_fn=None,
                                 biases_initializer=tf.zeros_initializer(),
                                 scope='fc8')
                end_points = slim.utils.convert_collection_to_dict(end_points_collection)
                if spatial_squeeze:
                    net = tf.squeeze(net, [1, 2], name='fc8/squeezed')
                    end_points[sc.name + '/fc8'] = net
                    return net, end_points
       
    alex_net.default_image_size=224